In [37]:
import pandas as pd
from bs4 import BeautifulSoup
import requests
import os
import finnhub
import pandas as pd
from datetime import datetime, timedelta
import yfinance as yf
import re
import os


In [38]:
from selenium import webdriver

from selenium.webdriver.chrome.service import Service
from selenium.webdriver.chrome.options import Options
from selenium.webdriver.common.by import By
import time
path = "/Users/lukasamare/Desktop/chromedriver-mac-arm64/chromedriver"

service= Service(executable_path=path)

In [39]:
# retrieving api key
path_to_api = '/Users/lukasamare/data'
filename = 'finnhub.txt'
api_key_path = os.path.join(path_to_api,filename)
with open(api_key_path, 'r') as file:
 finnhub_api_key = file.read().strip()
 #print("API:", finnhub_api_key)

In [40]:
#Webscraping code with Selenium
url =  "https://finance.yahoo.com/screener/predefined/undervalued_growth_stocks/"
options = Options()
options.binary_location = "/Users/lukasamare/Desktop/chrome-mac-arm64/Google Chrome for Testing.app/Contents/MacOS/Google Chrome for Testing"
driver = webdriver.Chrome(service = service, options = options)
#options.binary_location = os.path.expanduser("~/Desktop/Google Chrome.app/Contents/MacOS/Google Chrome")
driver.get(url)
element  = driver.find_elements(By.XPATH, "//*[@data-test='quoteLink']")
ticker_lst = []
for ticker in element:
    ticker_lst.append(ticker.text)
print(ticker_lst)

driver.quit()


['CVE', 'LYG', 'LAUR', 'AEO', 'BKR', 'CVS', 'MTCH', 'JWN', 'MGM', 'SYF', 'DOLE', 'NOV', 'PGR', 'HMY', 'GS', 'FOXA', 'APLE', 'URBN', 'SLM', 'ACGL', 'BBVA', 'FNF', 'ARCC', 'GPN', 'USA']


In [41]:
print(len(ticker_lst))

25


In [42]:

finnhub_client = finnhub.Client(api_key=finnhub_api_key)


In [43]:
# Time for API calls 2 months from today
today = datetime.today()

two_years = today -timedelta(days=365*2)
two_months_ago = today-timedelta(days=60)
two_months = str(two_months_ago).replace("-","").replace(" ","T")[:14].replace(":","")

print(two_months)

20240821T2216


In [44]:
today = str(today).split()[0]
two_years = str(two_years).split()[0]
d = finnhub_client.stock_insider_transactions("DAL",two_years,today)
#filters data into a data frame  with top 3 transaction values
def filter_data_trans(data):
  pos_trans = [elem for elem in data["data"] if elem["change"]>0]
  df = pd.DataFrame(pos_trans)
 
  df["transaction_value"]= df["change"]*df["transactionPrice"]
  top3 = df["transaction_value"].sort_values(ascending=False)[:3]
  filt = df[df["transaction_value"].isin(top3)]
  

  return filt.set_index("name")
filter_data_trans(d)


,change,currency,filingDate,id,isDerivative,share,source,symbol,transactionCode,transactionDate,transactionPrice,transaction_value
name,,,,,,,,,,,,
Chiang Willie CW,10000,,2024-05-03,0001683168-24-002965,False,10000,sec,DAL,P,2024-05-01,49.4955,494955.00
Bastian Edward H,71840,,2024-02-02,0001683168-24-000652,False,500608,sec,DAL,M,2024-02-01,30.8900,2219137.60
BLAKE FRANCIS S,12880,,2023-01-20,0001683168-23-000263,False,93795,sec,DAL,P,2023-01-18,38.5795,496903.96


In [45]:
#merges transactions into one dataframe
df_lst = []
for ticks in ticker_lst:
    call = finnhub_client.stock_insider_transactions(ticks,two_years,today)
    if call["data"]==[]:
       print(ticks)
    else:
        df_lst.append(filter_data_trans(call))
        top3_instrans_alltickers = pd.concat(df_lst)
#push this data frame into snowflake
top3_instrans_alltickers

#make dataframe where it shows ticker and max transaction over 200,000 
#category column with all other data 


LYG
DOLE
HMY


/var/folders/8l/kydfwjds0pdf_9nnjjth36t80000gn/T/ipykernel_5665/3123664847.py:9: FutureWarning: The behavior of DataFrame concatenation with empty or all-NA entries is deprecated. In a future version, this will no longer exclude empty or all-NA columns when determining the result dtypes. To retain the old behavior, exclude the relevant entries before the concat operation.
  top3_instrans_alltickers = pd.concat(df_lst)
/var/folders/8l/kydfwjds0pdf_9nnjjth36t80000gn/T/ipykernel_5665/3123664847.py:9: FutureWarning: The behavior of DataFrame concatenation with empty or all-NA entries is deprecated. In a future version, this will no longer exclude empty or all-NA columns when determining the result dtypes. To retain the old behavior, exclude the relevant entries before the concat operation.
  top3_instrans_alltickers = pd.concat(df_lst)
/var/folders/8l/kydfwjds0pdf_9nnjjth36t80000gn/T/ipykernel_5665/3123664847.py:9: FutureWarning: The behavior of DataFrame concatenation with empty or all-NA

,change,currency,filingDate,id,isDerivative,share,source,symbol,transactionCode,transactionDate,transactionPrice,transaction_value
name,,,,,,,,,,,,
Cenovus Energy Inc.,2700399,CAD,2024-10-10,,None,2700399.0,sedi,CVE.TO,38,2024-09-26,22.1400,59786833.86
Cenovus Energy Inc.,3233900,CAD,2024-09-10,,None,3233900.0,sedi,CVE.TO,38,2024-08-20,25.8500,83596315.00
Cenovus Energy Inc.,1912700,CAD,2023-01-10,,None,1912700.0,sedi,CVE.TO,38,2022-12-05,26.3400,50380518.00
Knauer Gerard M.,1965,,2024-06-05,0001415889-24-015657,False,17933.0,sec,LAUR,M,2024-06-03,7.6400,15012.60
Sinkfield Richard Harvey III,12692,,2023-08-11,0001415889-23-012171,False,77863.0,sec,LAUR,M,2023-08-09,8.3400,105851.28
...,...,...,...,...,...,...,...,...,...,...,...,...
SLOAN JEFFREY STEVEN,100764,,2023-02-23,0001127602-23-006865,True,100764.0,sec,GPN,A,2023-02-21,113.1200,11398423.68
SLOAN JEFFREY STEVEN,41991,,2023-02-23,0001127602-23-006865,False,314529.0,sec,GPN,A,2023-02-21,113.1200,4750021.92
BURKE EDMUND J,4000,,2024-03-26,0001281236-24-000001,False,6500.0,sec,USA,P,2024-03-26,7.0859,28343.60


In [46]:
tvh_ticker_lst = []
for i in range(len(top3_instrans_alltickers)):
    if top3_instrans_alltickers.iloc[i]["transaction_value"] >200000:
        tvh_ticker_lst.append(top3_instrans_alltickers.iloc[i]["symbol"])
tvh_ticker_lst = list(set(tvh_ticker_lst))
print(tvh_ticker_lst)  





['SLM', 'FNF', 'JWN', 'BKR', 'AEO', 'MGM', 'CVS', 'MTCH', 'APLE', 'CVE.TO', 'SYF', 'FOXA', 'URBN', 'BBVA.MC', 'ACGL', 'ARCC', 'GPN']


In [48]:
import re
path = "/Users/lukasamare/Desktop/chromedriver-mac-arm64/chromedriver"
service= Service(executable_path=path)
options = Options()
options.binary_location = "/Users/lukasamare/Desktop/chrome-mac-arm64/Google Chrome for Testing.app/Contents/MacOS/Google Chrome for Testing"
driver = webdriver.Chrome(service = service, options = options)

urllst = [ "https://finance.yahoo.com/quote/"+ticker+"/key-statistics/" for ticker in tvh_ticker_lst]
print(urllst)
sir_data = []
#getting all the urls in a list from the cleaned insider transaction ticker list
for ticker in tvh_ticker_lst:
    #getting all the urls in a list from the cleaned insider transaction ticker list
    print(ticker)
    url = "https://finance.yahoo.com/quote/"+ticker+"/key-statistics/"
    driver.get(url)
    element  = driver.find_elements(By.XPATH, "//td[@class='value yf-vaowmx']")
    statslst = []
    #appending all the stats from all the rows int a list 
    for stats in element:
         statslst.append(stats.text)
    # print(statslst)
    shortlst ={"Ticker": ticker}
    if re.match(r"^\d", statslst[37]):
      shortlst["Short_Interest_Rate"] = float(statslst[37])
    else:
        shortlst["Short_Interest_Rate"] = None
    sir_data.append(shortlst)

driver.quit()
    


['https://finance.yahoo.com/quote/SLM/key-statistics/', 'https://finance.yahoo.com/quote/FNF/key-statistics/', 'https://finance.yahoo.com/quote/JWN/key-statistics/', 'https://finance.yahoo.com/quote/BKR/key-statistics/', 'https://finance.yahoo.com/quote/AEO/key-statistics/', 'https://finance.yahoo.com/quote/MGM/key-statistics/', 'https://finance.yahoo.com/quote/CVS/key-statistics/', 'https://finance.yahoo.com/quote/MTCH/key-statistics/', 'https://finance.yahoo.com/quote/APLE/key-statistics/', 'https://finance.yahoo.com/quote/CVE.TO/key-statistics/', 'https://finance.yahoo.com/quote/SYF/key-statistics/', 'https://finance.yahoo.com/quote/FOXA/key-statistics/', 'https://finance.yahoo.com/quote/URBN/key-statistics/', 'https://finance.yahoo.com/quote/BBVA.MC/key-statistics/', 'https://finance.yahoo.com/quote/ACGL/key-statistics/', 'https://finance.yahoo.com/quote/ARCC/key-statistics/', 'https://finance.yahoo.com/quote/GPN/key-statistics/']
SLM
FNF
JWN
BKR
AEO
MGM
CVS
MTCH
APLE
CVE.TO
SYF
FO

In [49]:
sir_data

[{'Ticker': 'SLM', 'Short_Interest_Rate': 4.61},
 {'Ticker': 'FNF', 'Short_Interest_Rate': 3.37},
 {'Ticker': 'JWN', 'Short_Interest_Rate': 4.18},
 {'Ticker': 'BKR', 'Short_Interest_Rate': 2.4},
 {'Ticker': 'AEO', 'Short_Interest_Rate': 3.45},
 {'Ticker': 'MGM', 'Short_Interest_Rate': 2.95},
 {'Ticker': 'CVS', 'Short_Interest_Rate': 2.27},
 {'Ticker': 'MTCH', 'Short_Interest_Rate': 3.52},
 {'Ticker': 'APLE', 'Short_Interest_Rate': 5.33},
 {'Ticker': 'CVE.TO', 'Short_Interest_Rate': 1.84},
 {'Ticker': 'SYF', 'Short_Interest_Rate': 3.68},
 {'Ticker': 'FOXA', 'Short_Interest_Rate': 6.21},
 {'Ticker': 'URBN', 'Short_Interest_Rate': 2.82},
 {'Ticker': 'BBVA.MC', 'Short_Interest_Rate': None},
 {'Ticker': 'ACGL', 'Short_Interest_Rate': 3.41},
 {'Ticker': 'ARCC', 'Short_Interest_Rate': 3.92},
 {'Ticker': 'GPN', 'Short_Interest_Rate': 2.52}]

In [51]:
#float(statslst[37])
sirdf = pd.DataFrame(sir_data)

max_transaction = top3_instrans_alltickers[["symbol","transaction_value"]].groupby("symbol").max().reset_index()

max_transaction.rename(columns={'symbol': 'Ticker'}, inplace=True)
max_transaction

,Ticker,transaction_value
0,ACGL,1.540991e+06
1,AEO,3.325601e+06
2,APLE,3.769824e+06
3,ARCC,2.676000e+05
4,BBVA.MC,3.281442e+05
5,BKR,1.803746e+06
6,CVE.TO,8.359632e+07
7,CVS,1.820238e+07
8,FNF,2.800164e+05
9,FOXA,4.783859e+06


In [ ]:
#I will get sentiment from top3 rescent articles
# I will avg the sentiment score
# I will obtain label off of sentiment score

In [ ]:
combinedf = sirdf.merge(right = max_transaction, how = "inner", on ="Ticker")


def rank_trans(x):
   
    if x <200000:
        return "small transaction"
    if x>=200000 and x<1000000:
        return "medium transaction"
    if x>=1000000:
        return "large transaction"

combinedf["transaction_value"] = combinedf["transaction_value"].apply(rank_trans)
combinedf

,Ticker,Short_Interest_Rate,transaction_value
0,ELD.TO,9.98,large transaction
1,ARCC,5.73,medium transaction
2,MGM,4.13,large transaction
3,MTCH,4.33,large transaction
4,URBN,2.41,medium transaction
5,APLE,5.81,large transaction
6,FOXA,8.97,large transaction
7,ACGL,4.28,large transaction
8,SYF,4.99,large transaction
9,BKR,3.04,large transaction


In [ ]:

def scrape_articles(top3articles):
 sentiment_scores =[]
 mean_sent_scr = None
 for i in top3articles:
     i["ticker_sentiment"]
     a = i["ticker_sentiment"]
     for j in a:
         if j["ticker"] == ticker and float(j["relevance_score"]) > 0.5:
            sentiment_scores.append(float(j["ticker_sentiment_score"]))
            mean_sent_scr = sum(sentiment_scores)/len(sentiment_scores)
 if mean_sent_scr is not None:
  if mean_sent_scr <= -0.35:
     return "Bearish"
  if mean_sent_scr > -0.35 and mean_sent_scr <= -0.15:
     return "Somewhat-Bearish"
  if mean_sent_scr > -0.15 and mean_sent_scr < 0.15:
     return "Neutral"
  if mean_sent_scr >= -0.15 and mean_sent_scr < 0.35:
     return "Somewhat-Bullish"
  if mean_sent_scr >=0.35:
     return "Bullish"
 else:
    return "low relevance score"


In [ ]:
#alphaquerey = "https://www.alphavantage.co/query?function=NEWS_SENTIMENT&tickers=AAPL&time_from="+two_months+"&sort=LATEST&limit=2&apikey=SAIGXA9UVDHR0EAQ"
# r  = requests.get(alphaquerey)
# data = r.json()
#top3articles  = data["feed"][:3]
ticker_sentiment =[]
print(tvh_ticker_lst)
for ticker in tvh_ticker_lst:
    print(ticker)
    query = "https://www.alphavantage.co/query?function=NEWS_SENTIMENT&tickers="+ticker+"&time_from="+two_months+"&sort=LATEST&limit=2&apikey=SAIGXA9UVDHR0EAQ"
    r  = requests.get(query)
    data = r.json()
    top3articles  = data["feed"][:3]
    sentiment = {"Ticker":ticker}
    sentiment['sentiment'] =  scrape_articles(top3articles)
    ticker_sentiment.append(sentiment)
masterdf =pd.DataFrame(ticker_sentiment)
masterdf = combinedf.merge(right = masterdf, on="Ticker")


    



['ELD.TO', 'ARCC', 'MGM', 'MTCH', 'URBN', 'APLE', 'FOXA', 'ACGL', 'SYF', 'BKR', 'SLM', 'FNF', 'SU.TO', 'AEO', 'CVE.TO', 'BBVA.MC', 'JWN', 'CVS', 'DAL']
ELD.TO


KeyError: 'feed'

In [ ]:
#masterdf

tic_prac = tvh_ticker_lst[10:]
print(data)
print(tvh_ticker_lst[0])
print(two_months)
for i in tvh_ticker_lst:
  query = "https://www.alphavantage.co/query?function=NEWS_SENTIMENT&tickers="+i+"&time_from="+two_months+"&sort=LATEST&limit=2&apikey=SAIGXA9UVDHR0EAQ"
  r  = requests.get(query)
  data = r.json()

combinedf
top3_instrans_alltickers.set_index("symbol")

{'Information': 'Thank you for using Alpha Vantage! Our standard API rate limit is 25 requests per day. Please subscribe to any of the premium plans at https://www.alphavantage.co/premium/ to instantly remove all daily rate limits.'}
ELD.TO
20240802T2054


KeyboardInterrupt: 

In [ ]:
print()

[{'title': 'CROWDSTRIKE ALERT: Bragar Eagel & Squire, P.C. is Investigating CrowdStrike Holdings, Inc. on Behalf of Long-Term Stockholders and Encourages Investors to Contact the Firm - CrowdStrike Holdings  ( NASDAQ:CRWD ) ', 'url': 'https://www.benzinga.com/pressreleases/24/10/g41133565/crowdstrike-alert-bragar-eagel-squire-p-c-is-investigating-crowdstrike-holdings-inc-on-behalf-of-l', 'time_published': '20241002T010000', 'authors': ['Globe Newswire'], 'summary': 'NEW YORK, Oct. 01, 2024 ( GLOBE NEWSWIRE ) -- Bragar Eagel & Squire, P.C., a nationally recognized shareholder rights law firm, is investigating potential claims against CrowdStrike Holdings, Inc.', 'banner_image': 'https://www.benzinga.com/next-assets/images/schema-image-default.png', 'source': 'Benzinga', 'category_within_source': 'News', 'source_domain': 'www.benzinga.com', 'topics': [{'topic': 'Energy & Transportation', 'relevance_score': '0.5'}, {'topic': 'Technology', 'relevance_score': '0.5'}, {'topic': 'Financial Ma

In [ ]:
filtered_master = masterdf[(masterdf["transaction_value"]=="medium transaction")&(masterdf["transction_value"]=="large transaction")&(masterdf["Short_Interest_Rate"]>5)]

In [ ]:
#practice on 1 ticker
#BKR
import requests
from datetime import datetime, timedelta
ticker_sentiment =[]
def scrape_articles(top3articles):
 sentiment_scores =[]
 mean_sent_scr = None
 for i in top3articles:
     i["ticker_sentiment"]
     a = i["ticker_sentiment"]
     for j in a:
         if j["ticker"] == ticker and float(j["relevance_score"]) > 0.5:
            sentiment_scores.append(float(j["ticker_sentiment_score"]))
            mean_sent_scr = sum(sentiment_scores)/len(sentiment_scores)
 if mean_sent_scr is not None:
  if mean_sent_scr <= -0.35:
     return "Bearish"
  if mean_sent_scr > -0.35 and mean_sent_scr <= -0.15:
     return "Somewhat-Bearish"
  if mean_sent_scr > -0.15 and mean_sent_scr < 0.15:
     return "Neutral"
  if mean_sent_scr >= -0.15 and mean_sent_scr < 0.35:
     return "Somewhat-Bullish"
  if mean_sent_scr >=0.35:
     return "Bullish"
 else:
    return "low relevance score"

today = datetime.today()

two_months_ago = today-timedelta(days=60)

two_months = str(two_months_ago).replace("-","").replace(" ","T")[:14].replace(":","")

for ticker in ["BKR"]:
    query = "https://www.alphavantage.co/query?function=NEWS_SENTIMENT&tickers="+ticker+"&time_from="+two_months+"&sort=LATEST&limit=2&apikey=SAIGXA9UVDHR0EAQ"
    r  = requests.get(query)
    data = r.json()
    top3articles  = data["feed"][:3]
    sentiment = {"Ticker":ticker}
    sentiment['sentiment'] =  scrape_articles(top3articles)
    ticker_sentiment.append(sentiment)


In [ ]:
# print(ticker_sentiment)
# masterdf =pd.DataFrame(ticker_sentiment)
# masterdf = combinedf.merge(right = masterdf, on="Ticker")
# masterdf

print(data)

{'Information': 'Invalid inputs. Please refer to the API documentation https://www.alphavantage.co/documentation#newsapi and try again.'}


In [ ]:
# #BKR
# import requests
# from datetime import datetime, timedelta
# lst = ["BKR"]
# len(lst)
# today = datetime.today()

# two_months_ago = today-timedelta(days=60)

# two_months = str(two_months_ago).replace("-","").replace(" ","T")[:14].replace(":","")
# query = "https://www.alphavantage.co/query?function=NEWS_SENTIMENT&tickers=BKR&time_from="+two_months+"&sort=LATEST&limit=2&apikey=SAIGXA9UVDHR0EAQ"
# r  = requests.get(query)
# data = r.json()
# top3articles  = data["feed"][:3]
# top3articles

[{'title': 'C3.ai Stock: A Tale of Two Entities',
  'url': 'https://www.fool.com/investing/2024/10/01/c3ai-stock-a-tale-of-two-entities/',
  'time_published': '20241001T115100',
  'authors': ['Will Healy'],
  'summary': 'Does the potential of its AI outweigh its financial challenges?',
  'banner_image': 'https://g.foolcdn.com/editorial/images/792154/ai-chip.jpg',
  'source': 'Motley Fool',
  'category_within_source': 'n/a',
  'source_domain': 'www.fool.com',
  'topics': [{'topic': 'Earnings', 'relevance_score': '0.99676'},
   {'topic': 'Technology', 'relevance_score': '1.0'},
   {'topic': 'Financial Markets', 'relevance_score': '0.459462'}],
  'overall_sentiment_score': 0.188213,
  'overall_sentiment_label': 'Somewhat-Bullish',
  'ticker_sentiment': [{'ticker': 'BKR',
    'relevance_score': '0.134968',
    'ticker_sentiment_score': '0.121655',
    'ticker_sentiment_label': 'Neutral'}]},
 {'title': 'Crude Oil Rises; Onconetix Shares Spike Higher - Onconetix  ( NASDAQ:ONCO ) ',
  'url': 

In [ ]:
# def scrape_articles(top3articles):
#  sentiment_scores =[]
#  mean_sent_scr = None
#  for i in top3articles:
#      i["ticker_sentiment"]
#      a = i["ticker_sentiment"]
#      for j in a:
#          if j["ticker"] == ticker and float(j["relevance_score"]) > 0.5:
#             sentiment_scores.append(float(j["ticker_sentiment_score"]))
#             mean_sent_scr = sum(sentiment_scores)/len(sentiment_scores)
#  if mean_sent_scr is not None:
#   if mean_sent_scr <= -0.35:
#      return "Bearish"
#   if mean_sent_scr > -0.35 and mean_sent_scr <= -0.15:
#      return "Somewhat-Bearish"
#   if mean_sent_scr > -0.15 and mean_sent_scr < 0.15:
#      return "Neutral"
#   if mean_sent_scr >= -0.15 and mean_sent_scr < 0.35:
#      return "Somewhat-Bullish"
#   if mean_sent_scr >=0.35:
#      return "Bullish"
#  else:
#     return "low relevance score"
# print(scrape_articles(top3articles))

low relevance score
